# Lab9: Lending Dataset – Regularized Regression


#### Author: Dianna Hummel

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

C:\Users\Dianna\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


##### Step A

In [2]:
loan_data = pd.read_csv("loansDataClean.csv", index_col = 'ID')
loan_data.head(10)

,IntRate,LoanAmt,LoanTerm,Debt2Inc,Home__MORT,Home__OWN,Home__RENT,MonthlyInc,RevCredBal,FICO,EmpLen
ID,,,,,,,,,,,
81174,8.90,20000,0,14.90,1,0,0,6541.67,14272.0,737,0
99592,12.12,19200,0,28.36,1,0,0,4583.33,11140.0,717,2
80059,21.98,35000,1,23.81,1,0,0,11500.00,21977.0,692,2
33182,11.71,12000,0,18.78,0,0,1,3195.00,14469.0,697,9
62403,15.31,6000,0,20.05,0,1,0,4891.67,10391.0,672,3
48808,7.90,10000,0,26.09,0,0,1,2916.67,15957.0,722,10
76404,14.33,14675,0,26.92,0,0,1,3150.00,7246.0,687,8
15867,6.91,7000,0,7.10,0,0,1,5000.00,7612.0,717,3
94971,19.72,2000,0,10.29,0,0,1,3575.00,12036.0,672,6


In [3]:
loan_data.shape

(1752, 11)

In [4]:
loan_data.dtypes

IntRate       float64
LoanAmt         int64
LoanTerm        int64
Debt2Inc      float64
Home__MORT      int64
Home__OWN       int64
Home__RENT      int64
MonthlyInc    float64
RevCredBal    float64
FICO            int64
EmpLen          int64
dtype: object

In [5]:
loan_data.describe().T

,count,mean,std,min,25%,50%,75%,max
IntRate,1752.0,13.120811,4.200848,5.42,10.1600,13.110,15.800,24.89
LoanAmt,1752.0,11609.474886,7443.034671,1000.00,6000.0000,10000.000,15000.000,35000.00
LoanTerm,1752.0,0.183790,0.387423,0.00,0.0000,0.000,0.000,1.00
Debt2Inc,1752.0,15.706062,7.632795,0.00,9.8275,15.625,21.165,34.91
Home__MORT,1752.0,0.434361,0.495814,0.00,0.0000,0.000,1.000,1.00
Home__OWN,1752.0,0.083904,0.277323,0.00,0.0000,0.000,0.000,1.00
Home__RENT,1752.0,0.481735,0.499809,0.00,0.0000,0.000,1.000,1.00
MonthlyInc,1752.0,5520.837854,3186.290186,1000.00,3416.6700,4833.330,6666.670,39583.33
RevCredBal,1752.0,13932.856164,13708.658521,0.00,5539.2500,10738.500,17680.000,143151.00
FICO,1752.0,704.568493,33.186834,662.00,677.0000,697.000,722.000,832.00


#### Step B

In [6]:
Y = loan_data [['IntRate']]
X = loan_data.drop(['IntRate'], axis=1)

In [7]:
sc = StandardScaler()
sc.fit(X)
X_std = sc.transform(X)
X_std[:5]

array([[ 1.1276208 , -0.4745259 , -0.1056352 ,  1.14115464, -0.30263639,
        -0.9641138 ,  0.32047414,  0.02474645,  0.97751917, -1.53355396],
       [ 1.02010707, -0.4745259 ,  1.65831149,  1.14115464, -0.30263639,
        -0.9641138 , -0.29431579, -0.20378752,  0.37469848, -0.96624262],
       [ 3.14350321,  2.10736655,  1.06202936,  1.14115464, -0.30263639,
        -0.9641138 ,  1.87706358,  0.58696046, -0.37882739, -0.96624262],
       [ 0.05248351, -0.4745259 ,  0.40284275, -0.87630543, -0.30263639,
         1.03722195, -0.73016008,  0.03912104, -0.22812221,  1.0193471 ],
       [-0.75386945, -0.4745259 ,  0.56927754, -0.87630543,  3.30429529,
        -0.9641138 , -0.19751732, -0.25844012, -0.98164808, -0.68258694]])

In [8]:
X_std = sm.add_constant (X_std)
loan_model0 = sm.OLS (Y, X_std)
loan_model0 = loan_model0.fit()
loan_model0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                IntRate   R-squared:                       0.754
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     593.5
Date:                Wed, 01 Nov 2017   Prob (F-statistic):               0.00
Time:                        15:51:32   Log-Likelihood:                -3771.3
No. Observations:                1752   AIC:                             7563.
Df Residuals:                    1742   BIC:                             7617.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.1208      0.050    262.957      0.000      13.023      13.219
x1             1.0979      0.062     17.676      0.000       0.976       1.220
x2             1.3223      0.054     24.460      0.000       1.216       1.428
x3            -0.1068      0.055     -1.930      0.054      -0.215       0.002
x4            -0.0873      0.031     -2.829      0.005      -0.148      -0.027
x5             0.0467      0.043      1.075      0.282      -0.038       0.132
x6             0.0607      0.030      2.035      0.042       0.002       0.119
x7            -0.0994      0.063     -1.568      0.117      -0.224       0.025
x8            -0.0919      0.060     -1.532      0.126      -0.209       0.026
x9            -3.1247      0.052    -60.423      0.000      -3.226      -3.023
x10            0.0402      0.052      0.772      0.440      -0.062       0.142
==============================================================================
Omnibus:                      112.621   Durbin-Watson:                   1.934
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              187.144
Skew:                           0.492   Prob(JB):                     2.30e-41
Kurtosis:                       4.263   Cond. No.                     6.28e+15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.1e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [9]:
X_std1 = np.delete(X_std, [10], axis=1)

In [10]:
X_std1
X_std0 = sm.add_constant (X_std1)
loan_model0 = sm.OLS (Y, X_std0)
loan_model0 = loan_model0.fit()
loan_model0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                IntRate   R-squared:                       0.754
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     667.8
Date:                Wed, 01 Nov 2017   Prob (F-statistic):               0.00
Time:                        15:51:32   Log-Likelihood:                -3771.6
No. Observations:                1752   AIC:                             7561.
Df Residuals:                    1743   BIC:                             7610.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.1208      0.050    262.988      0.000      13.023      13.219
x1             1.1026      0.062     17.842      0.000       0.981       1.224
x2             1.3224      0.054     24.465      0.000       1.216       1.428
x3            -0.1075      0.055     -1.943      0.052      -0.216       0.001
x4            -0.0823      0.030     -2.728      0.006      -0.141      -0.023
x5             0.0460      0.043      1.059      0.290      -0.039       0.131
x6             0.0561      0.029      1.921      0.055      -0.001       0.113
x7            -0.1011      0.063     -1.596      0.111      -0.225       0.023
x8            -0.0890      0.060     -1.487      0.137      -0.206       0.028
x9            -3.1254      0.052    -60.451      0.000      -3.227      -3.024
==============================================================================
Omnibus:                      111.850   Durbin-Watson:                   1.934
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              185.978
Skew:                           0.489   Prob(JB):                     4.12e-41
Kurtosis:                       4.262   Cond. No.                     5.58e+15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.34e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [11]:
X_std1 = np.delete(X_std, [5,10], axis=1)

In [12]:
X_std0 = sm.add_constant (X_std1)
loan_model0 = sm.OLS (Y, X_std0)
loan_model0 = loan_model0.fit()
loan_model0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                IntRate   R-squared:                       0.754
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     667.8
Date:                Wed, 01 Nov 2017   Prob (F-statistic):               0.00
Time:                        15:51:32   Log-Likelihood:                -3771.6
No. Observations:                1752   AIC:                             7561.
Df Residuals:                    1743   BIC:                             7610.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.1208      0.050    262.988      0.000      13.023      13.219
x1             1.1026      0.062     17.842      0.000       0.981       1.224
x2             1.3224      0.054     24.465      0.000       1.216       1.428
x3            -0.1075      0.055     -1.943      0.052      -0.216       0.001
x4            -0.1645      0.095     -1.738      0.082      -0.350       0.021
x5            -0.0267      0.093     -0.286      0.775      -0.210       0.157
x6            -0.1011      0.063     -1.596      0.111      -0.225       0.023
x7            -0.0890      0.060     -1.487      0.137      -0.206       0.028
x8            -3.1254      0.052    -60.451      0.000      -3.227      -3.024
==============================================================================
Omnibus:                      111.850   Durbin-Watson:                   1.934
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              185.978
Skew:                           0.489   Prob(JB):                     4.12e-41
Kurtosis:                       4.262   Cond. No.                         3.93
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
X_std1 = np.delete(X_std, [5,6,10], axis=1)
X_std0 = sm.add_constant (X_std1)
loan_model0 = sm.OLS (Y, X_std0)
loan_model0 = loan_model0.fit()
loan_model0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                IntRate   R-squared:                       0.754
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     763.6
Date:                Wed, 01 Nov 2017   Prob (F-statistic):               0.00
Time:                        15:51:32   Log-Likelihood:                -3771.6
No. Observations:                1752   AIC:                             7559.
Df Residuals:                    1744   BIC:                             7603.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.1208      0.050    263.057      0.000      13.023      13.219
x1             1.1026      0.062     17.846      0.000       0.981       1.224
x2             1.3223      0.054     24.470      0.000       1.216       1.428
x3            -0.1071      0.055     -1.937      0.053      -0.216       0.001
x4            -0.1419      0.052     -2.717      0.007      -0.244      -0.039
x5            -0.1015      0.063     -1.604      0.109      -0.226       0.023
x6            -0.0888      0.060     -1.484      0.138      -0.206       0.029
x7            -3.1247      0.052    -60.521      0.000      -3.226      -3.023
==============================================================================
Omnibus:                      111.730   Durbin-Watson:                   1.934
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              185.466
Skew:                           0.489   Prob(JB):                     5.33e-41
Kurtosis:                       4.259   Cond. No.                         2.33
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
X_std1 = np.delete(X_std, [5,6,8,10], axis=1)
X_std0 = sm.add_constant (X_std1)
loan_model0 = sm.OLS (Y, X_std0)
loan_model0 = loan_model0.fit()
loan_model0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                IntRate   R-squared:                       0.754
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     889.9
Date:                Wed, 01 Nov 2017   Prob (F-statistic):               0.00
Time:                        15:51:32   Log-Likelihood:                -3772.7
No. Observations:                1752   AIC:                             7559.
Df Residuals:                    1745   BIC:                             7598.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.1208      0.050    262.966      0.000      13.023      13.219
x1             1.0884      0.061     17.824      0.000       0.969       1.208
x2             1.3228      0.054     24.472      0.000       1.217       1.429
x3            -0.1315      0.053     -2.490      0.013      -0.235      -0.028
x4            -0.1498      0.052     -2.883      0.004      -0.252      -0.048
x5            -0.1362      0.059     -2.314      0.021      -0.252      -0.021
x6            -3.1242      0.052    -60.492      0.000      -3.226      -3.023
==============================================================================
Omnibus:                      109.817   Durbin-Watson:                   1.934
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              181.065
Skew:                           0.484   Prob(JB):                     4.81e-40
Kurtosis:                       4.242   Cond. No.                         2.06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
scores_list = []
for rep in range(128):
    lr = LinearRegression()
    scores = cross_val_score(lr, X_std0, Y, cv=8)
    scores_list.append(scores.mean())
print ('Mean R-Squared Score:', sum (scores_list)/len (scores_list))

Mean R-Squared Score: 0.749959588063


In [16]:
lr = LinearRegression()
X_train, X_test, Y_train, Y_test = train_test_split(X_std0,Y,random_state=33,test_size=0.2)
model = lr.fit (X_train, Y_train)
Y_pred = lr.predict (X_test)
preds = Y_pred[:5]
preds

array([[ 15.64255519],
       [  6.59548795],
       [ 13.82204776],
       [  7.82483241],
       [ 13.8250073 ]])

In [17]:
Y_test[:5].values

array([[ 14.33],
       [  5.79],
       [ 13.67],
       [  8.9 ],
       [ 13.49]])

In [18]:
model.coef_

array([[ 0.        ,  1.08346743,  1.29230826, -0.0921673 , -0.15510927,
        -0.15126294, -3.12942754]])

In [19]:
model.intercept_

array([ 13.13233964])

In [20]:
model.score (X_test, Y_test)

0.78339610037504681

In [21]:
metrics.mean_squared_error (Y_test, Y_pred)

4.2308403805289378

In [22]:
lr = LinearRegression()
X_train, X_test, Y_train, Y_test = train_test_split(X_std0,
Y, random_state=3,test_size=0.2)
model = lr.fit(X_train, Y_train)
predictions = lr.predict(X_test)
print(predictions[:5])
print(Y_test[:5].values)
print(model.coef_)
print(model.intercept_)
print(metrics.mean_squared_error (Y_test, predictions))
print('R-Squared Score:', model.score(X_test, Y_test))

[[ 11.06552759]
 [ 13.77971589]
 [ 12.59468905]
 [  6.80612659]
 [  9.26459797]]
[[  8.49]
 [ 16.29]
 [ 14.09]
 [  6.62]
 [  7.88]]
[[ 0.          1.06963748  1.2823738  -0.13660673 -0.10849553 -0.13872206
  -3.16959378]]
[ 13.11982758]
4.68010707286
R-Squared Score: 0.748995385127


In [23]:
lr = LinearRegression()
X_train, X_test, Y_train, Y_test = train_test_split(X_std0,
Y, random_state=13,test_size=0.2)
model = lr.fit(X_train, Y_train)
predictions = lr.predict(X_test)
print(predictions[:5])
print(Y_test[:5].values)
print(model.coef_)
print(model.intercept_)
print(metrics.mean_squared_error (Y_test, predictions))
print('R-Squared Score:', model.score(X_test, Y_test))

[[ 13.64000738]
 [ 14.45046533]
 [ 12.04869505]
 [ 13.85777977]
 [ 15.35512902]]
[[ 14.09]
 [ 17.77]
 [ 11.49]
 [ 14.09]
 [ 17.77]]
[[ 0.          1.04677445  1.3520537  -0.17063715 -0.13697887 -0.1260454
  -3.17444375]]
[ 13.13996531]
4.22698563799
R-Squared Score: 0.746591861628


In [24]:
lr = LinearRegression()
X_train, X_test, Y_train, Y_test = train_test_split(X_std0,
Y, random_state=23,test_size=0.2)
model = lr.fit(X_train, Y_train)
predictions = lr.predict(X_test)
print(predictions[:5])
print(Y_test[:5].values)
print(model.coef_)
print(model.intercept_)
print(metrics.mean_squared_error (Y_test, predictions))
print('R-Squared Score:', model.score(X_test, Y_test))

[[ 13.73591382]
 [ 15.07196615]
 [ 15.18774707]
 [ 10.45677232]
 [ 15.09663893]]
[[ 16.29]
 [ 13.67]
 [ 17.77]
 [  7.9 ]
 [ 14.61]]
[[ 0.          1.12237293  1.3481246  -0.16711445 -0.15438311 -0.21764075
  -3.19576666]]
[ 13.14614919]
4.58784571637
R-Squared Score: 0.713570896221


##### Step C

In [25]:
Y = loan_data [['IntRate']]
X = loan_data.drop(['IntRate','EmpLen','Home__OWN','Home__RENT','RevCredBal'], axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=33,test_size=0.2)

ridgemod = Ridge (alpha=0.0, normalize=True)
ridgemod.fit (X_train, Y_train)
Y_pred = ridgemod.predict (X_test)

In [26]:
preds = Y_pred[:5]
preds

array([[ 15.64255519],
       [  6.59548795],
       [ 13.82204776],
       [  7.82483241],
       [ 13.8250073 ]])

In [27]:
acts = Y_test[:5].values
acts

array([[ 14.33],
       [  5.79],
       [ 13.67],
       [  8.9 ],
       [ 13.49]])

In [28]:
ridgemod.coef_

array([[  1.45609529e-04,   3.33660094e+00,  -1.20786187e-02,
         -3.12926748e-01,  -4.74866136e-05,  -9.43241836e-02]])

In [29]:
ridgemod.intercept_

array([ 77.87430015])

In [30]:
print ('R-Squared Score:', ridgemod.score(X_test, Y_test))

R-Squared Score: 0.783396100375


In [31]:
print ("Ridge alpha=0 MSE", metrics.mean_squared_error (Y_test, Y_pred))

Ridge alpha=0 MSE 4.23084038053


##### Step D

In [32]:
alpha_range = [0.01, 0.1, 0.25, 0.5, 0.75, 1, 2.5, 5, 7.5, 10, 25, 50, 75, 100]
print ("RidgeCV alpha array", alpha_range) 

RidgeCV alpha array [0.01, 0.1, 0.25, 0.5, 0.75, 1, 2.5, 5, 7.5, 10, 25, 50, 75, 100]


In [33]:
ridcvmod = RidgeCV (alphas=alpha_range, scoring='neg_mean_squared_error')
ridcvmod.fit (X_train, Y_train)
Y_pred = ridcvmod.predict (X_test)
Y_pred[:5]

array([[ 15.64766429],
       [  6.59570624],
       [ 13.82361021],
       [  7.82581218],
       [ 13.82494325]])

In [34]:
print("RidgeCV Regression Best Alpha", ridcvmod.alpha_)

RidgeCV Regression Best Alpha 0.75


In [35]:
print("RidgeCV Regression Coefficients", ridcvmod.coef_)

RidgeCV Regression Coefficients [[  1.45913684e-04   3.32225703e+00  -1.21249224e-02  -3.11975886e-01
   -4.77493741e-05  -9.43326036e-02]]


In [36]:
print ("RidgeCV Regression Test R-squared", ridcvmod.score (X_test, Y_test))

RidgeCV Regression Test R-squared 0.7833206523


In [37]:
print ("RidgeCV Regression MSE", metrics.mean_squared_error (Y_pred, Y_test))

RidgeCV Regression MSE 4.23231407865


##### Step E

In [38]:
Y = loan_data [['IntRate']]
X = loan_data.drop(['IntRate','EmpLen','Home__OWN',
 'Home__RENT','RevCredBal'], axis=1)
Y = Y.values
Y = Y.ravel()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
random_state=33,test_size=0.2)

In [39]:
lascvmod = LassoCV (n_alphas=100, normalize=True,
 random_state=33)
lascvmod.fit(X_train, Y_train)
Y_pred = lascvmod.predict(X_test)
Y_pred[:5]


array([ 15.54853096,   6.82117272,  13.82631855,   7.73871915,  13.92975875])

In [40]:
print("LassoCV Regression Best Alpha", lascvmod.alpha_)

LassoCV Regression Best Alpha 0.00123305262621


In [41]:
print("LassoCV Regression Coefficients", lascvmod.coef_)

LassoCV Regression Coefficients [  1.33554702e-04   3.28515373e+00  -1.76193619e-03  -2.53930698e-01
  -2.11660218e-05  -9.26951262e-02]


In [42]:
print ("LassoCV Regression Test R-squared",
 lascvmod.score (X_test, Y_test))


LassoCV Regression Test R-squared 0.781579380462


In [43]:
print ("LassoCV Regression MSE",
 metrics.mean_squared_error (Y_pred, Y_test)) 

LassoCV Regression MSE 4.26632566948


##### Step F

In [44]:
Y = loan_data [['IntRate']]
X = loan_data.drop(['IntRate'], axis=1)
Y = Y.values
Y = Y.ravel()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
random_state=33,test_size=0.2)

In [45]:
lascvmod = LassoCV (n_alphas=100, normalize=True,
 random_state=33)
lascvmod.fit(X_train, Y_train)
Y_pred = lascvmod.predict(X_test)
Y_pred[:5]


array([ 15.51382685,   6.89377613,  13.77691187,   7.7324606 ,  13.97114691])

In [46]:
print("LassoCV Regression Best Alpha", lascvmod.alpha_)

LassoCV Regression Best Alpha 0.00174782023918


In [47]:
print("LassoCV Regression Coefficients", lascvmod.coef_)

LassoCV Regression Coefficients [  1.29658624e-04   3.26121752e+00  -0.00000000e+00  -2.19507287e-01
   9.12452316e-03   0.00000000e+00  -9.04833708e-06  -1.72881333e-06
  -9.21549379e-02   0.00000000e+00]


In [48]:
print ("LassoCV Regression Test R-squared",
 lascvmod.score (X_test, Y_test))

LassoCV Regression Test R-squared 0.781260836356


In [49]:
print ("LassoCV Regression MSE",
 metrics.mean_squared_error (Y_pred, Y_test))

LassoCV Regression MSE 4.27254766859
